In [ ]:
# %% md
# # 01  Simulation & Calibration
# Simulate Hawkes order-flow → estimate Hawkes MLE → fit propagator kernel
# %%
import numpy as np, pandas as pd, matplotlib.pyplot as plt, os
os.makedirs("../docs/figures", exist_ok=True)
# %% 1. simulate
import sys; sys.path.append("..")
from sim.simulate_hawkes import simulate_hawkes_ogata
from sim.simulate_orderflow import simulate_orderbook_sequence
T = 3_600  # 1h
mu, alpha, beta = 0.1, 0.5, 1.0
times, _ = simulate_hawkes_ogata(T, mu, [alpha], [beta], seed=42)
print(f"{len(times)} events")
# %% 2. Hawkes MLE
from calib.hawkes_mle import hawkes_mle
params, stderr = hawkes_mle(times, kernel="exp")
mu_hat, alpha_hat, beta_hat = params['mu'][0], params['alpha'][0], params['beta'][0]
print(f"MLE: μ={mu_hat:.3f}±{stderr['mu'][0]:.3f}  α={alpha_hat:.3f}±{stderr['alpha'][0]:.3f}  β={beta_hat:.3f}±{stderr['beta'][0]:.3f}")
# %% 3. order-flow trace
hawkes_p = dict(mu_vec=np.array([mu_hat, mu_hat]),
                A=np.array([[0.2,0.15],[0.15,0.2]]),
                betas=np.array([[2,2],[2,2]]))
prop_p = dict(permanent=0.0,
              transient=dict(alphas=np.array([5e-4,2e-4]),
                             betas=np.array([10.,50.])),
              rho=5.0)
trace = simulate_orderbook_sequence(T, hawkes_p, prop_p,
                                    dict(name='lognormal',mean=10,std=5),
                                    initial_price=100, dt_ms=1, seed=43)
trace.to_csv("../docs/figures/sample_orderflow.csv", index=False)
print("order-flow saved")
# %% 4. propagator estimation
from calib.propagator_estimation import estimate_propagator_regression, fit_multi_exponential
lags, Gnon = estimate_propagator_regression(trace, max_lag=60)
alphas, rhos, info = fit_multi_exponential(lags, Gnon, n_exp=2)
print("fitted propagator:", alphas, rhos)
# save
pd.DataFrame({'lag':lags, 'G':Gnon}).to_csv("../docs/figures/kernel_nonpar.csv", index=False)
import pickle, os
with open("../docs/figures/kernel_fit.pkl",'wb') as f: pickle.dump({'alphas':alphas,'rhos':rhos, **info}, f)
# %% 5. kernel plot
t fine = np.linspace(0, lags[-1], 200)
Gfit = sum(a*np.exp(-r*t_fine) for a,r in zip(alphas,rhos))
plt.figure(figsize=(4,2.5))
plt.plot(lags, Gnon*1e4, 'o', ms=3, label='non-parametric')
plt.plot(t_fine, Gfit*1e4, '-', label='multi-exp fit')
plt.xlabel("lag (s)"); plt.ylabel(r"$G(\tau)\,(10^{-4})$")
plt.legend(); plt.grid(alpha=.2)
plt.tight_layout(); plt.savefig("../docs/figures/kernel_fit.pdf")
plt.show()